In [1]:
!pip install -U transformers

In [2]:
import pandas as pd
from tqdm import tqdm

## Local Inference on GPU
Model page: https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-Coder-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am Qwen, an AI language model developed by Alibaba Cloud. My purpose is to assist users in generating human-like text based on the input I receive, which can be used for a variety of tasks such as answering questions, providing explanations, generating creative content, and more. How can I help you today?'}]}]

In [4]:
# Settings
MODEL_ID = 'Qwen/Qwen2.5-Coder-7B-Instruct'
INPUT_CSV = 'malicious_code_prompts.csv'
OUTPUT_CSV = 'malicious_code_results.csv'

In [5]:
# Load prompts and run batch inference
prompts = pd.read_csv(INPUT_CSV)
results = []

for _, row in tqdm(prompts.iterrows(), total=len(prompts)):
    idx = row['idx']
    prompt = row['goal']
    try:
        result = pipe(prompt)
    except Exception as e:
        result = f'ERROR: {e}'
    results.append({'idx': idx, 'result': result})

100%|██████████| 82/82 [07:38<00:00,  5.59s/it]


In [6]:
# Save results
out_df = pd.DataFrame(results)
out_df.to_csv(OUTPUT_CSV, index=False)
print(f'Saved results to {OUTPUT_CSV}')

Saved results to malicious_code_results.csv


fix, don't use sequential

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))